Testing Propotype with Selenium & google library
*** I have using FireFox for the selenium web driver. Be sure that Firefox has been installed on the local machine.

In [ ]:
pip install selenium pandas google


In [6]:
#from googlesearch import search_news
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urlparse
import pandas as pd
import time
import traceback
import re

In [8]:
OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3
SLEEP_INTERVAL = 2

p_elementList= []

# Cuntion for Getting Date from Content
def extractDate(string):
    try:
       # Define a regular expression for the date format DD/MM/YYYY and optional time
        date_time_pattern = re.compile(
            r"^\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b"  # Matches dates like DD/MM/YYYY at the start of the string
            r"(?:\s+\b\d{1,2}:\d{2}(?::\d{2})?\s*(?:AM|PM)?\s*(?:ET|WT)?\b)?$",  # Optionally matches times with optional AM/PM and time zone
            re.IGNORECASE  # Make the pattern case-insensitive
        )
        # Check if the entire string matches the pattern
        match = date_time_pattern.match(string)
        return bool(match)
    except:
        print("Error accessing a text:", e)
    return False

# Function for Getting Content from url
def getContent(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        date = ''
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
            
        # Getting Title
        for h1 in H1_elements:
            try:
                title = h1.text
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                if value.lower().strip().startswith('written by:'):
                    authors = value.lower().replace('written by:', '') 
                    #authors = value.lower().replace('by ', '') 
                    #words = authors.split()
                    #if len(words) <= AUTHOR_MAX_WORDS and not authors:
                    #    authors = value
                    #else:
                    #    authors = ''
                    continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue

                # call date function to extract the date
                if (value.lower().strip().startswith('updated on')):
                    dateValue = value.lower().replace('updated on', '') 
                    if (extractDate(dateValue)):
                        date = dateValue
                        continue

                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'date' : date, 'title' : title, 'content': paragraph, 'source_domain': source_domain, 'url' : url, 'copyright': copyright } 

def getPElementList(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        readMoreUrl = ''
        date = ''
        driver.get(url)
        p_elements = driver.find_elements(By.TAG_NAME, "p")

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                p_elementList.append({'p_element': value,'title' : title,  'source_domain': source_domain, 'url' : url, })

            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return 

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')


In [9]:
# define query String
#queryString = 'Did Joe Biden tell voters to stay home and not vote?'
#queryString = 'Is Michael Saylor saying he will give away free bitcoin?'
#queryString = 'Is Taylor Swift going to give away a free Le Creuset cookware set?'
#queryString = 'Did Elon Musk say you can control a computer mouse with neuralink?'
#queryString = 'Oprah winfrey weight loss gummies?'
#queryString = 'Did Justin Trudeau say invest in Petro canada?'
queryString = 'How to enter Jennifer Aniston MacBook giveaway?'

#Search in google url with google library
results = search(query=queryString,num = 10 )

In [10]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        data = getContent(driver,url)
        print(data)
        if (data['title'] !='' or data['content'] != ''):
            dataSet.append(data)
    except:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(dataSet)


Scripting from => https://malwaretips.com/blogs/jennifer-aniston-macbook-giveaway-scam/
{'authors': ' thomas orsolya', 'date': '', 'title': 'Don’t Get Scammed By The Fake Jennifer Aniston MacBook Giveaway', 'content': 'Published on: December 20, 2023 A fraudulent online campaign claiming beloved actress Jennifer Aniston is gifting free MacBook laptops has recently infiltrated social media through manipulated video ads and questionable website portals. However, the supposed giveaway actually intends stealing personal information and money rather than magnanimous charity. This article will break down exactly how the devious scam ensnares victims step-by-step alongside protection tips to avoid potential damages.   This Article Contains: A fraudulent online campaign claiming universally-beloved comedy icon Jennifer Aniston has teamed up with Apple facilitating an epic giveaway gifting free MacBook laptops aims exploiting fans through deception and psychological manipulation rather than rea

In [18]:
df

,authors,date,title,content,source_domain,url,copyright
0,,,Don’t Get Scammed By The Fake Jennifer Aniston...,Written by: Thomas Orsolya Published on: Decem...,malwaretips.com,https://malwaretips.com/blogs/jennifer-aniston...,
1,,,Don’t Get Scammed By The Fake Jennifer Aniston...,Written by: Thomas Orsolya Published on: Decem...,malwaretips.com,https://malwaretips.com/blogs/jennifer-aniston...,
2,,,Don’t Get Scammed By The Fake Jennifer Aniston...,Written by: Thomas Orsolya Published on: Decem...,malwaretips.com,https://malwaretips.com/blogs/jennifer-aniston...,
3,,,"You're eligible! Click ""Get Offer"" below and C...",,www.pinterest.com,https://www.pinterest.com/pin/youre-eligible-c...,
4,,,Unveiling the Deceptive Jennifer Aniston MacBo...,"Updated on Feb 17,2024 In...",www.toolify.ai,https://www.toolify.ai/ai-news/unveiling-the-d...,Copyright ©2024 toolify
...,...,...,...,...,...,...,...
141,,,The Morning Show,A bold proposal to shore up a financially-chal...,tv.apple.com,https://tv.apple.com/nz/show/the-morning-show/...,Copyright © 2024 Apple Inc. All Rights Reserved.
142,,,"Global hit series “The Morning Show,” starring...",On the heels of the buzzy sophomore season fin...,www.apple.com,https://www.apple.com/tv-pr/news/2022/01/globa...,
143,,,"Global hit series “Morning Wars,” starring and...",On the heels of the buzzy season two finale of...,www.apple.com,https://www.apple.com/au/tv-pr/news/2022/01/gl...,
144,,,"""Jennifer Aniston‪""‬\nSmartLess",44 min The one-and-only Jennifer Aniston joi...,podcasts.apple.com,https://podcasts.apple.com/us/podcast/jennifer...,


In [19]:
save_to_csv_with_pandas(df, queryString.replace('?','.')+ 'csv')